In [254]:
import pandas as pd
import numpy as np
import json
import tqdm

handmade_game_ids_crosschecked.xlsx, game_ids.xlsx, crawling_result_with_codenames.xlsx 있어야 함.

In [255]:
strcolumn_dict = {"tournamentId":"str", "matchId":"str", "gameId":"str", "patch":"str"}
temp1 = [f"esportsPlayerId_{i}" for i in range(10)]
temp2 = ["str" for i in range(10)]
esportsPlayerId_type_dict = dict(zip(temp1, temp2))
strcolumn_dict.update(esportsPlayerId_type_dict)
game_ids_df = pd.read_excel("../data/game_ids.xlsx", dtype = strcolumn_dict)
#game_ids_df = game_ids_df.astype({"tournamentId":"str", "matchId":"str", "gameId":"str",})
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9287 entries, 0 to 9286
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         9287 non-null   object
 1   tournamentId             9287 non-null   object
 2   blockName                9287 non-null   object
 3   leagueName               9287 non-null   object
 4   matchId                  9287 non-null   object
 5   bestof                   9287 non-null   int64 
 6   blueteam_name            9287 non-null   object
 7   blueteam_code            9287 non-null   object
 8   blueteam_win             9287 non-null   int64 
 9   redteam_name             9287 non-null   object
 10  redteam_code             9287 non-null   object
 11  redteam_win              9287 non-null   int64 
 12  gameNumberInAMatch       9287 non-null   int64 
 13  gameId                   9287 non-null   object
 14  patch                    8081 non-null  

In [256]:
game_ids_df.head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_0,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9
0,2024-05-01T08:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,99322214662601038,101389713966873995,98767991798919851,108369186353082038,106625308523122120,104573202011010010,101971954398227333,98767991764579452,98767975924333750,99566406444168246
1,2024-05-01T08:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,99322214662601038,101389713966873995,98767991798919851,108369186353082038,106625308523122120,104573202011010010,101971954398227333,98767991764579452,98767975924333750,99566406444168246
2,2024-05-01T08:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,104573202011010010,101971954398227333,98767991764579452,98767975924333750,99566406444168246,99322214662601038,101389713966873995,98767991798919851,108369186353082038,106625308523122120
3,2024-05-01T11:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,...,105320680474347057,105320682452092471,98767991747728851,103495716775975785,103495716561790834,105516185566739968,99566408350815597,102825747663199158,106276221575238438,105506922297942200
4,2024-05-01T11:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,...,105516185566739968,99566408350815597,102825747663199158,106276221575238438,105506922297942200,105320680474347057,105320682452092471,98767991747728851,103495716775975785,103495716561790834


In [257]:
game_ids_with_picks_df = game_ids_df[pd.isna(game_ids_df["pick_0"]) == False].reset_index(drop=True)
game_ids_with_picks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8023 entries, 0 to 8022
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         8023 non-null   object
 1   tournamentId             8023 non-null   object
 2   blockName                8023 non-null   object
 3   leagueName               8023 non-null   object
 4   matchId                  8023 non-null   object
 5   bestof                   8023 non-null   int64 
 6   blueteam_name            8023 non-null   object
 7   blueteam_code            8023 non-null   object
 8   blueteam_win             8023 non-null   int64 
 9   redteam_name             8023 non-null   object
 10  redteam_code             8023 non-null   object
 11  redteam_win              8023 non-null   int64 
 12  gameNumberInAMatch       8023 non-null   int64 
 13  gameId                   8023 non-null   object
 14  patch                    8023 non-null  

In [258]:
game_ids_with_picks_df.tail()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_0,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9
8018,2023-04-22T10:00:00Z,109749565997987636,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,SBTC ESPORTS,...,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,107251636998679351,107255592262151792,107251642747641853,107251647709372417,102849319864045953
8019,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,107330029738468255,98767975918707483,107251572267658009,107251575502514974,102849319860638077
8020,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,107330029738468255,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935
8021,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,107330029738468255,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935
8022,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,107330029738468255,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935


In [259]:
def danpan_winnerside(row):
    if row["bestof"] == 1:
        return "Blue" if row["blueteam_win"] == 1 else "Red"
    else:
        return np.nan
game_ids_with_picks_df["winner_side"] = game_ids_with_picks_df.apply(danpan_winnerside, axis=1)
game_ids_with_picks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8023 entries, 0 to 8022
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         8023 non-null   object
 1   tournamentId             8023 non-null   object
 2   blockName                8023 non-null   object
 3   leagueName               8023 non-null   object
 4   matchId                  8023 non-null   object
 5   bestof                   8023 non-null   int64 
 6   blueteam_name            8023 non-null   object
 7   blueteam_code            8023 non-null   object
 8   blueteam_win             8023 non-null   int64 
 9   redteam_name             8023 non-null   object
 10  redteam_code             8023 non-null   object
 11  redteam_win              8023 non-null   int64 
 12  gameNumberInAMatch       8023 non-null   int64 
 13  gameId                   8023 non-null   object
 14  patch                    8023 non-null  

In [260]:
game_ids_with_picks_df.loc[game_ids_with_picks_df["bestof"] == 1].head()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side
153,2022-05-10T08:00:00Z,107905284983470149,Round 1,MSI,108176841597230180,1,T1,T1,1,Vikings Esports,...,105320682452092471,98767991747728851,103495716775975785,103495716561790834,107251528271309568,107251529370048921,107251531720338181,107251561724960718,107251533987191201,Blue
154,2022-05-10T09:00:00Z,107905284983470149,Round 1,MSI,108176841597230190,1,Team Aze,AZE,0,DetonatioN FocusMe,...,106105043525540824,100118511917058602,105320665816491252,99566405793083799,98767975944639514,100754278890207800,99566404536983507,100285330144336367,104016425624023728,Red
155,2022-05-10T10:00:00Z,107905284983470149,Round 1,MSI,108176841597295766,1,ORDER,ORD,0,G2 Esports,...,104668539132702977,105709404500072628,103524841906404187,101383792848062619,99566406047571736,99124844325223302,98767975968177297,103461966879623046,99322214657019865,Red
156,2022-05-10T11:00:00Z,107905284983470149,Round 1,MSI,108176841597295764,1,Evil Geniuses LG,EG,0,G2 Esports,...,101389713966873995,105504920899018806,105913361921383562,99101098208825497,99566406047571736,99124844325223302,98767975968177297,103461966879623046,99322214657019865,Red
157,2022-05-10T13:00:00Z,107905284983470149,Round 1,MSI,108176841597295748,1,RED Kalunga,RED,1,PSG Talon,...,103743617977630492,103743619023893329,99566407783597200,99566408211536727,99566406483827119,105320699390649600,105320700529018429,98767975916831837,98767975956663486,Blue


In [261]:
game_ids_with_picks_df.to_excel("../data/game_ids_with_winside_danpan.xlsx")

Exception ignored in: <function ZipFile.__del__ at 0x000001F3EA474EA0>
Traceback (most recent call last):
  File "g:\Program\Anaconda3\envs\newdeal\Lib\zipfile\__init__.py", line 1940, in __del__
    self.close()
  File "g:\Program\Anaconda3\envs\newdeal\Lib\zipfile\__init__.py", line 1957, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [262]:
game_ids_without_danpan_df = game_ids_with_picks_df[pd.isna(game_ids_with_picks_df["winner_side"]) == True].copy()
game_ids_without_danpan_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4498 entries, 0 to 8022
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         4498 non-null   object
 1   tournamentId             4498 non-null   object
 2   blockName                4498 non-null   object
 3   leagueName               4498 non-null   object
 4   matchId                  4498 non-null   object
 5   bestof                   4498 non-null   int64 
 6   blueteam_name            4498 non-null   object
 7   blueteam_code            4498 non-null   object
 8   blueteam_win             4498 non-null   int64 
 9   redteam_name             4498 non-null   object
 10  redteam_code             4498 non-null   object
 11  redteam_win              4498 non-null   int64 
 12  gameNumberInAMatch       4498 non-null   int64 
 13  gameId                   4498 non-null   object
 14  patch                    4498 non-null   obje

In [263]:
crawled_df = pd.read_excel("../data/crawling_result_with_codenames.xlsx")
crawled_df = crawled_df.astype({"tournament_id":"str", "patch":"str"})
crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20179 entries, 0 to 20178
Data columns (total 41 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              20179 non-null  object
 1   tournament_title  20179 non-null  object
 2   patch             20179 non-null  object
 3   blueteam          20179 non-null  object
 4   redteam           20179 non-null  object
 5   winner_side       20179 non-null  object
 6   ban_0             20135 non-null  object
 7   ban_1             20152 non-null  object
 8   ban_2             20147 non-null  object
 9   ban_3             16973 non-null  object
 10  ban_4             16887 non-null  object
 11  ban_5             20127 non-null  object
 12  ban_6             20151 non-null  object
 13  ban_7             20143 non-null  object
 14  ban_8             16978 non-null  object
 15  ban_9             16875 non-null  object
 16  pick_0            20179 non-null  object
 17  pick_1      

In [264]:
crawled_df_test = crawled_df[crawled_df["ban_0"].isna()]
crawled_df_test.head()

,date,tournament_title,patch,blueteam,redteam,winner_side,ban_0,ban_1,ban_2,ban_3,...,summonerName_5,summonerName_6,summonerName_7,summonerName_8,summonerName_9,tournament_name,tournament_id,tournament_slug,blueteam_code,redteam_code
2345,2016-01-30,\n\t\t\t\t\tNA LCS 2016 Spring,6.1,Team Impulse,Dignitas,Blue,NaN,NaN,Elise,NaN,...,BillyBoss,Kirei,Shiphtur,Apollo,KiWiKiD,NA LCS 2016 Spring,106972812538132507,na_lcs_spring_2016,TIP,DIG
2428,2015-07-04,\n\t\t\t\t\tNA LCS 2015 Summer,5.12,Cloud9,Team Liquid,Red,NaN,NaN,Sivir,NaN,...,Quas,IWDominate,FeniX,Piglet,Xpecial,NA LCS 2015 Summer,107741488941545959,na_lcs_summer_2015,C9,TL
2449,2015-06-20,\n\t\t\t\t\tNA LCS 2015 Summer,5.11,Team Dragon Knights,Counter Logic Gaming,Red,NaN,NaN,NaN,NaN,...,ZionSpartan,Xmithie,Pobelter,Doublelift,Aphromoo,NA LCS 2015 Summer,107741488941545959,na_lcs_summer_2015,TDK,CLG
2471,2015-05-31,\n\t\t\t\t\tNA LCS 2015 Summer,5.9,Team Dragon Knights,Gravity (North American Team),Red,NaN,NaN,NaN,NaN,...,Hauntzer,Move,Keane,Altec,Bunny FuFuu,NA LCS 2015 Summer,107741488941545959,na_lcs_summer_2015,TDK,GRV
2479,2015-05-30,\n\t\t\t\t\tNA LCS 2015 Summer,5.9,Team Dragon Knights,Team Liquid,Red,NaN,NaN,NaN,NaN,...,Quas,IWDominate,FeniX,Piglet,Xpecial,NA LCS 2015 Summer,107741488941545959,na_lcs_summer_2015,TDK,TL


In [265]:
pick_column_list = [f"pick_{x}" for x in range(10)]
left_on = ["tournamentId", "teamcode_blue_on_detail", "teamcode_red_on_detail"]
right_on = ["tournament_id", "blueteam_code", "redteam_code"]
left_on.extend(pick_column_list)
right_on.extend(pick_column_list)
game_ids_with_picks_df = game_ids_df.astype({"patch":"str"})
print(left_on)
print(right_on)




['tournamentId', 'teamcode_blue_on_detail', 'teamcode_red_on_detail', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']
['tournament_id', 'blueteam_code', 'redteam_code', 'pick_0', 'pick_1', 'pick_2', 'pick_3', 'pick_4', 'pick_5', 'pick_6', 'pick_7', 'pick_8', 'pick_9']


In [266]:
game_ids_without_danpan_df[left_on].head()

,tournamentId,teamcode_blue_on_detail,teamcode_red_on_detail,pick_0,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9
0,112099354464733424,FLY,PSG,Urgot,Sejuani,Taliyah,Jinx,Nautilus,Rumble,JarvanIV,Azir,Senna,TahmKench
1,112099354464733424,FLY,PSG,Urgot,Maokai,Taliyah,Lucian,Nami,KSante,Sejuani,Azir,Kalista,Ashe
2,112099354464733424,PSG,FLY,KSante,XinZhao,Karma,Kalista,Rell,Olaf,Sejuani,Orianna,Varus,Nautilus
3,112099354464733424,T1,EST,KSante,LeeSin,Orianna,Kalista,Neeko,Yone,Volibear,Ahri,Draven,Rell
4,112099354464733424,EST,T1,Aatrox,LeeSin,Ahri,Draven,Ashe,KSante,Viego,Orianna,Kalista,Rumble


In [267]:
crawled_df[right_on].head()

,tournament_id,blueteam_code,redteam_code,pick_0,pick_1,pick_2,pick_3,pick_4,pick_5,pick_6,pick_7,pick_8,pick_9
0,112099354464733424,GEN,BLG,Rumble,Lee Sin,Azir,Senna,Maokai,Camille,Nidalee,Tristana,Zeri,Nautilus
1,112099354464733424,BLG,GEN,Jax,Nidalee,Jayce,Senna,Tahm Kench,Rumble,Lee Sin,Hwei,Kalista,Ashe
2,112099354464733424,BLG,GEN,Camille,Xin Zhao,Orianna,Varus,Ashe,Zac,Sejuani,Yone,Kalista,Blitzcrank
3,112099354464733424,GEN,BLG,K'Sante,Karthus,Yone,Senna,Tahm Kench,Twisted Fate,Xin Zhao,Neeko,Lucian,Nami
4,112099354464733424,BLG,T1,Camille,Xin Zhao,Neeko,Senna,Ornn,K'Sante,Viego,Taliyah,Varus,Nautilus


In [268]:
with open('../data/pick_name_not_the_same.json', 'r') as f:
    pick_difference_map = json.load(f)
crawled_df[pick_column_list] = crawled_df[pick_column_list].replace(pick_difference_map)

In [269]:
picks_of_game_ids_set = set(np.concatenate(game_ids_with_picks_df.loc[:,pick_column_list].values.tolist()).tolist())
picks_of_crawled_set = set(np.concatenate(crawled_df.loc[:,pick_column_list].values.tolist()).tolist())
difference_of_game_ids = picks_of_game_ids_set.difference(picks_of_crawled_set)
difference_of_crawled = picks_of_crawled_set.difference(picks_of_game_ids_set)

In [270]:
print(difference_of_game_ids)
print(len(difference_of_game_ids))
print(difference_of_crawled)
print(len(difference_of_crawled))

{'nan'}
1
{'Master Yi'}
1


In [271]:
game_ids_without_danpan_df.tail()

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,...,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side
8018,2023-04-22T10:00:00Z,109749565997987636,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,SBTC ESPORTS,...,102848965907008705,102300322396525713,105301884027266192,98767975954252935,107251636998679351,107255592262151792,107251642747641853,107251647709372417,102849319864045953,NaN
8019,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,102848965907008705,102300322396525713,105301884027266192,98767975954252935,107330029738468255,98767975918707483,107251572267658009,107251575502514974,102849319860638077,NaN
8020,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,NaN
8021,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,NaN
8022,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,NaN


In [272]:
game_ids_without_danpan_df.drop("winner_side", axis=1, inplace = True)
game_ids_without_danpan_df.index.name = "index"
game_ids_without_danpan_df_reset = game_ids_without_danpan_df.reset_index()
result_df = pd.merge(game_ids_without_danpan_df_reset, crawled_df, how = "left", left_on = left_on, right_on = right_on).drop_duplicates("gameId")
result_df.set_index("index")

,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code_x,blueteam_win,redteam_name,...,summonerName_5_y,summonerName_6_y,summonerName_7_y,summonerName_8_y,summonerName_9_y,tournament_name,tournament_id,tournament_slug,blueteam_code_y,redteam_code_y
index,,,,,,,,,,,,,,,,,,,,,
0,2024-05-01T08:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,Azhi,JunJia,Maple,Betty,Woody,2024 Mid-Season Invitational,112099354464733424,msi_2024,FLY,PSG
1,2024-05-01T08:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,Azhi,JunJia,Maple,Betty,Woody,2024 Mid-Season Invitational,112099354464733424,msi_2024,FLY,PSG
2,2024-05-01T08:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,...,Bwipo,Inspired,Jensen,Massu,Busio,2024 Mid-Season Invitational,112099354464733424,msi_2024,PSG,FLY
3,2024-05-01T11:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,...,Zothve,Josedeodo,Cody,Snaker,Ackerman,2024 Mid-Season Invitational,112099354464733424,msi_2024,T1,EST
4,2024-05-01T11:00:00Z,112099354464733424,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,...,Zeus,Oner,Faker,Gumayusi,Keria,2024 Mid-Season Invitational,112099354464733424,msi_2024,EST,T1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,2023-04-22T10:00:00Z,109749565997987636,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,SBTC ESPORTS,...,Pun,Draktharr,Yado,Artemis,Hieu3,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,SE,CES
8019,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,Kiaya,Levi,Kati,Sty1e,Zin,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,SE,GAM
8020,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,...,Nper,DNK,Dia1,Slayder,Palette,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,GAM,SE


In [273]:
result_df.tail()

,index,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code_x,blueteam_win,...,summonerName_5_y,summonerName_6_y,summonerName_7_y,summonerName_8_y,summonerName_9_y,tournament_name,tournament_id,tournament_slug,blueteam_code_y,redteam_code_y
4589,8018,2023-04-22T10:00:00Z,109749565997987636,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,...,Pun,Draktharr,Yado,Artemis,Hieu3,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,SE,CES
4590,8019,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,Kiaya,Levi,Kati,Sty1e,Zin,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,SE,GAM
4591,8020,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,Nper,DNK,Dia1,Slayder,Palette,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,GAM,SE
4592,8021,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,Nper,DNK,Dia1,Slayder,Palette,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,GAM,SE
4593,8022,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,Nper,DNK,Dia1,Slayder,Palette,VCS/2023 Season/Spring Season,109749565997987636,vcs_spring_2023,GAM,SE


In [274]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4498 entries, 0 to 4593
Data columns (total 79 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    4498 non-null   int64 
 1   startTime(match)         4498 non-null   object
 2   tournamentId             4498 non-null   object
 3   blockName                4498 non-null   object
 4   leagueName               4498 non-null   object
 5   matchId                  4498 non-null   object
 6   bestof                   4498 non-null   int64 
 7   blueteam_name            4498 non-null   object
 8   blueteam_code_x          4498 non-null   object
 9   blueteam_win             4498 non-null   int64 
 10  redteam_name             4498 non-null   object
 11  redteam_code_x           4498 non-null   object
 12  redteam_win              4498 non-null   int64 
 13  gameNumberInAMatch       4498 non-null   int64 
 14  gameId                   4498 non-null   obje

In [275]:
result_df.to_excel("../data/game_ids_merged.xlsx", index = False)

In [276]:
def concat_columns(row):
    return f"{row["blueteam_name"]} vs {row["redteam_name"]}"

In [277]:
handmade_df = result_df[pd.isna(result_df["winner_side"]) == True].copy()
handmade_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 883 entries, 164 to 4544
Data columns (total 79 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    883 non-null    int64 
 1   startTime(match)         883 non-null    object
 2   tournamentId             883 non-null    object
 3   blockName                883 non-null    object
 4   leagueName               883 non-null    object
 5   matchId                  883 non-null    object
 6   bestof                   883 non-null    int64 
 7   blueteam_name            883 non-null    object
 8   blueteam_code_x          883 non-null    object
 9   blueteam_win             883 non-null    int64 
 10  redteam_name             883 non-null    object
 11  redteam_code_x           883 non-null    object
 12  redteam_win              883 non-null    int64 
 13  gameNumberInAMatch       883 non-null    int64 
 14  gameId                   883 non-null    obj

In [278]:
columns = ["startTime(match)", "tournamentId", "teamcode_blue_on_detail", "teamcode_red_on_detail", "winner_side", "gameId", "patch_x", "blueteam_name", "redteam_name"]
columns.extend([f"pick_{i}" for i in range(10)])
columns.extend([f"summonerName_{i}_x" for i in range(10)])
handmade_df_simple = handmade_df.loc[:, columns]
handmade_df_simple["match_vs"] = handmade_df_simple.apply(concat_columns, axis = 1)
handmade_df_simple.drop(["blueteam_name", "redteam_name"], axis=1, inplace=True)
columns_order = ["startTime(match)", "tournamentId", "match_vs", "teamcode_blue_on_detail", "teamcode_red_on_detail", "winner_side", "gameId", "patch_x"]
columns_order.extend([f"pick_{i}" for i in range(10)])
columns_order.extend([f"summonerName_{i}_x" for i in range(10)])
handmade_df_simple = handmade_df_simple[columns_order]

In [279]:
handmade_df_simple.info()

<class 'pandas.core.frame.DataFrame'>
Index: 883 entries, 164 to 4544
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         883 non-null    object
 1   tournamentId             883 non-null    object
 2   match_vs                 883 non-null    object
 3   teamcode_blue_on_detail  883 non-null    object
 4   teamcode_red_on_detail   883 non-null    object
 5   winner_side              0 non-null      object
 6   gameId                   883 non-null    object
 7   patch_x                  883 non-null    object
 8   pick_0                   883 non-null    object
 9   pick_1                   883 non-null    object
 10  pick_2                   883 non-null    object
 11  pick_3                   883 non-null    object
 12  pick_4                   883 non-null    object
 13  pick_5                   883 non-null    object
 14  pick_6                   883 non-null    obj

In [280]:
handmade_df_simple.to_excel("../data/handmade_game_ids_simple.xlsx")

In [281]:
handmade_df.to_excel("../data/handmade_game_ids.xlsx")

여기서 우리가 수작업한 엑셀파일 handmade_game_ids_crosschecked.xlsx가 들어간다!

In [301]:
crosschecked_df = pd.read_excel("../data/handmade_game_ids_crosschecked.xlsx", dtype = "str")[["gameId", "winner_side"]]
crosschecked_df.columns = ["gameId", "winner_side_x"]
crosschecked_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gameId         883 non-null    object
 1   winner_side_x  883 non-null    object
dtypes: object(2)
memory usage: 13.9+ KB


In [302]:
result_winner_side = result_df[["gameId", "winner_side"]]
winnerside_of_merged = result_winner_side[result_winner_side["winner_side"].isna() == False]
winnerside_of_merged.columns = ["gameId", "winner_side_y"]
winnerside_of_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3615 entries, 0 to 4593
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gameId         3615 non-null   object
 1   winner_side_y  3615 non-null   object
dtypes: object(2)
memory usage: 84.7+ KB


In [333]:
game_ids_df = pd.read_excel("../data/game_ids_with_winside_danpan.xlsx", dtype = strcolumn_dict)
game_ids_df.tail()

,Unnamed: 0,startTime(match),tournamentId,blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,...,esportsPlayerId_1,esportsPlayerId_2,esportsPlayerId_3,esportsPlayerId_4,esportsPlayerId_5,esportsPlayerId_6,esportsPlayerId_7,esportsPlayerId_8,esportsPlayerId_9,winner_side
8018,8018,2023-04-22T10:00:00Z,109749565997987636,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,...,102848965907008705,102300322396525713,105301884027266192,98767975954252935,107251636998679351,107255592262151792,107251642747641853,107251647709372417,102849319864045953,NaN
8019,8019,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,102848965907008705,102300322396525713,105301884027266192,98767975954252935,107330029738468255,98767975918707483,107251572267658009,107251575502514974,102849319860638077,NaN
8020,8020,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,NaN
8021,8021,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,NaN
8022,8022,2023-04-23T10:00:00Z,109749565997987636,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,...,98767975918707483,107251572267658009,107251575502514974,102849319860638077,107251383938441076,102848965907008705,102300322396525713,105301884027266192,98767975954252935,NaN


In [334]:
game_ids_df = pd.merge(game_ids_df, winnerside_of_merged, how = "left", on = "gameId")
game_ids_df = pd.merge(game_ids_df, crosschecked_df, how = "left", on = "gameId")
game_ids_df.rename(columns = {"winner_side":"winner_side_z"}, inplace = True)
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8023 entries, 0 to 8022
Data columns (total 51 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               8023 non-null   int64 
 1   startTime(match)         8023 non-null   object
 2   tournamentId             8023 non-null   object
 3   blockName                8023 non-null   object
 4   leagueName               8023 non-null   object
 5   matchId                  8023 non-null   object
 6   bestof                   8023 non-null   int64 
 7   blueteam_name            8023 non-null   object
 8   blueteam_code            8023 non-null   object
 9   blueteam_win             8023 non-null   int64 
 10  redteam_name             8023 non-null   object
 11  redteam_code             8023 non-null   object
 12  redteam_win              8023 non-null   int64 
 13  gameNumberInAMatch       8023 non-null   int64 
 14  gameId                   8023 non-null  

In [335]:
def makeWinnerSideInAColumn(row):
    if pd.isna(row["winner_side_z"]) == False:
        row["winner_side"] = row["winner_side_z"]
    elif pd.isna(row["winner_side_y"]) == False:
        row["winner_side"] = row["winner_side_y"]
    else:
        row["winner_side"] = row["winner_side_x"]
    return row

In [336]:
game_ids_df = game_ids_df.apply(makeWinnerSideInAColumn, axis = 1)
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8023 entries, 0 to 8022
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               8023 non-null   int64 
 1   startTime(match)         8023 non-null   object
 2   tournamentId             8023 non-null   object
 3   blockName                8023 non-null   object
 4   leagueName               8023 non-null   object
 5   matchId                  8023 non-null   object
 6   bestof                   8023 non-null   int64 
 7   blueteam_name            8023 non-null   object
 8   blueteam_code            8023 non-null   object
 9   blueteam_win             8023 non-null   int64 
 10  redteam_name             8023 non-null   object
 11  redteam_code             8023 non-null   object
 12  redteam_win              8023 non-null   int64 
 13  gameNumberInAMatch       8023 non-null   int64 
 14  gameId                   8023 non-null  

In [337]:
game_ids_df.drop(["Unnamed: 0", "winner_side_x", "winner_side_y", "winner_side_z"], axis = 1, inplace = True)
game_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8023 entries, 0 to 8022
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   startTime(match)         8023 non-null   object
 1   tournamentId             8023 non-null   object
 2   blockName                8023 non-null   object
 3   leagueName               8023 non-null   object
 4   matchId                  8023 non-null   object
 5   bestof                   8023 non-null   int64 
 6   blueteam_name            8023 non-null   object
 7   blueteam_code            8023 non-null   object
 8   blueteam_win             8023 non-null   int64 
 9   redteam_name             8023 non-null   object
 10  redteam_code             8023 non-null   object
 11  redteam_win              8023 non-null   int64 
 12  gameNumberInAMatch       8023 non-null   int64 
 13  gameId                   8023 non-null   object
 14  patch                    8023 non-null  

In [338]:
game_ids_df.to_excel("../data/game_ids.xlsx")